In [18]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV

In [19]:
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()

In [20]:
# preparing the dataset
df = pd.DataFrame(housing.data, columns=housing.feature_names)


In [21]:
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [22]:
from urllib.parse import urlparse

In [23]:
x = df.drop('HouseAge', axis=1)
y = df['HouseAge']

In [24]:
def hyperparametertuning(X_train, y_train, params):
    rf = RandomForestRegressor()
    grid_search = GridSearchCV(estimator=rf,param_grid=params,cv=3, n_jobs=-1, verbose=2,
                               scoring="neg_mean_squared_error")
    grid_search.fit(X_train, y_train)
    return grid_search

In [27]:
# spliting the data in train test
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2)

from mlflow.models import infer_signature
signature = infer_signature(X_train, y_train)

# difine the hyper paprameters 
params = {
    'n_estimators': [100,200],             
    'max_depth': [5,10,None],             
    'min_samples_split': [2,5],        
    'min_samples_leaf': [1,2],                               
}

# mlflow experements
with mlflow.start_run():
    # perform hyperparamter tuning
    grid_search = hyperparametertuning(X_train, y_train, params)
    # get the best model
    best_model = grid_search.best_estimator_
    # evaluate the model 
    y_preds = best_model.predict(X_train)
    mse = mean_squared_error(y_train,y_preds)
    
    # logging the params 
    mlflow.log_param("best_n_estimator",grid_search.best_params_["n_estimators"])
    mlflow.log_param("best_max_depth",grid_search.best_params_["max_depth"])
    mlflow.log_param("best_min_samples_split",grid_search.best_params_["min_samples_split"])
    mlflow.log_param("best_min_samples_leaf",grid_search.best_params_["min_samples_leaf"])
    mlflow.log_metric("mse",mse)
    # tracking uri setup
    mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
    tracking_url_type_store =urlparse(mlflow.get_tracking_uri()).scheme
    if tracking_url_type_store != 'file':
        mlflow.sklearn.log_model(best_model,"model", registered_model_name="Best randomforest model")
    else:
        mlflow.sklearn.log_model(best_model, "model", signature=signature)
    

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   2.9s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   2.9s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   3.1s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   3.1s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   3.1s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   6.3s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   3.5s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   6.4s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   6.4s
[CV] END max_depth=5, min_samples_leaf=

2024/12/29 20:39:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'Best randomforest model'.
2024/12/29 20:39:02 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Best randomforest model, version 1


🏃 View run victorious-cat-678 at: http://127.0.0.1:5000/#/experiments/0/runs/b933e3ecabc34f2b964cb73d46274893
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0


Created version '1' of model 'Best randomforest model'.
